In [1]:
import sys
from pathlib import Path
import warnings

warnings.filterwarnings("ignore", module="IPython")

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    if root_dir.parts[-1:] == ("src",):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ("airquality",):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ("notebooks",):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir)
    print("Local environment")

print(f"Root dir: {root_dir}")

if root_dir not in sys.path:
    sys.path.append(root_dir)
    print(f"Added the following directory to the PYTHONPATH: {root_dir}")

from utils import config

settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Root dir: /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn
Added the following directory to the PYTHONPATH: /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn
HopsworksSettings initialized!


In [2]:
import datetime
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
import hopsworks
import json
from utils import airquality
import os

warnings.filterwarnings("ignore")

In [3]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store()

secrets = hopsworks.get_secrets_api()
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value

# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name="air_quality_all",
    version=1,
)
weather_fg = fs.get_feature_group(
    name="weather_all",
    version=1,
)

2025-11-17 17:26:55,018 INFO: Initializing external client
2025-11-17 17:26:55,018 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-17 17:26:56,543 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279179


Set SENSOR_CSV_FILE in .env with the relative path to a sensor to process it, or leave it unset to process all sensors in the `data` folder

In [4]:
sensor_csv_file = getattr(settings, 'SENSOR_CSV_FILE', None)

if sensor_csv_file:
    # Read one secret for single sensor mode
    _, _, _, _, _, sensor_id = airquality.read_sensor_data(sensor_csv_file)
    secret_name = f"SENSOR_LOCATION_JSON_{sensor_id}"
    location_str = secrets.get_secret(secret_name).value
    locations = {sensor_id: json.loads(location_str)}
else:
    # Read all individual secrets in batch mode
    all_secrets = secrets.get_secrets()
    locations = {}
    for secret in all_secrets:
        if secret.name.startswith("SENSOR_LOCATION_JSON_"):
            sensor_id = secret.name.replace("SENSOR_LOCATION_JSON_", "")
            location_str = secrets.get_secret(secret.name).value
            if location_str:
                locations[sensor_id] = json.loads(location_str)


## Helper Methods

In [5]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name="air_quality_all",
    version=1,
)
weather_fg = fs.get_feature_group(
    name="weather_all",
    version=1,
)

today = datetime.datetime.now().replace(tzinfo=None)
past_date = today - datetime.timedelta(days=4)

In [6]:
batch_weather = weather_fg.filter(weather_fg.date >= past_date).read()
batch_weather["date"] = pd.to_datetime(batch_weather["date"]).dt.tz_localize(None)
print(batch_weather.info())

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.75s) 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         150 non-null    datetime64[us]
 1   temperature_2m_mean          150 non-null    float32       
 2   precipitation_sum            150 non-null    float32       
 3   wind_speed_10m_max           150 non-null    float32       
 4   wind_direction_10m_dominant  150 non-null    float32       
 5   city                         150 non-null    object        
 6   sensor_id                    150 non-null    object        
 7   latitude                     150 non-null    float64       
 8   longitude                    150 non-null    float64       
dtypes: datetime64[us](1), float32(4), float64(2), object(2)
memory usage: 8.3+ KB
None


In [7]:
try:
    batch_airquality = air_quality_fg.filter(air_quality_fg.date >= past_date).read()
    batch_airquality["date"] = pd.to_datetime(batch_airquality["date"]).dt.tz_localize(None)
except Exception:
    batch_airquality = pd.DataFrame()
print(batch_airquality.info())

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.69s) 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             57 non-null     datetime64[us]
 1   pm25             57 non-null     float64       
 2   sensor_id        57 non-null     object        
 3   street           57 non-null     object        
 4   city             57 non-null     object        
 5   country          57 non-null     object        
 6   feed_url         57 non-null     object        
 7   pm25_rolling_3d  56 non-null     float64       
 8   pm25_lag_1d      56 non-null     float64       
 9   pm25_lag_2d      56 non-null     float64       
 10  pm25_lag_3d      56 non-null     float64       
 11  pm25_nearby_avg  57 non-null     float64       
dtypes: datetime64[us](1), float64(6), object(5)
memory usage: 5.5+ K

## Predictions

In [8]:
mr = project.get_model_registry()

MODEL_NAME_TEMPLATE = "air_quality_xgboost_model_{sensor_id}"

# model, model_dir, feature_view
retrieved_models = {}
skipped_sensors = []

for sensor_id in locations.keys():
    model_name = MODEL_NAME_TEMPLATE.format(sensor_id=sensor_id)
    retrieved_model = None

    available_models = mr.get_models(name=model_name)
    if available_models:
        retrieved_model = max(available_models, key=lambda model: model.version)

    saved_model_dir = retrieved_model.download()
    fv = retrieved_model.get_feature_view()

    retrieved_models[sensor_id] = retrieved_model, saved_model_dir, fv

if not retrieved_models:
    raise RuntimeError("No models were retrieved for the configured sensors.")

Downloading: 0.000%|          | 0/552706 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/24254 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/130270 elapsed<00:00 remaining<?

2025-11-17 17:27:11,040 INFO: There is no parent information


Downloading: 0.000%|          | 0/562426 elapsed<00:00 remaining<?

...

2025-11-17 17:28:00,781 INFO: There is no parent information


Downloading: 0.000%|          | 0/522579 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/31626 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/114290 elapsed<00:00 remaining<?

2025-11-17 17:28:04,297 INFO: There is no parent information


## Prediction

In [ ]:
# Merge historical data with weather data
batch_data = pd.merge(batch_weather, batch_airquality, on=["date", "sensor_id"], how="left")
batch_data = batch_data.sort_values(["sensor_id", "date"])

feature_cols = [
    "pm25_rolling_3d",
    "pm25_lag_1d",
    "pm25_lag_2d",
    "pm25_lag_3d",
    "pm25_nearby_avg",
]

batch_data["predicted_pm25"] = np.nan
batch_data["days_before_forecast_day"] = np.nan
for col in feature_cols:
    batch_data[f"predicted_{col}"] = np.nan

forecast_days = (
    batch_data.loc[batch_data["pm25"].isna(), "date"]
    .dropna()
    .sort_values()
    .unique()
)

model_cache = {}

for target_day in forecast_days:
    # context with all sensors up to current day
    window = batch_data.loc[batch_data["date"] <= target_day].copy()
    day_rows = window[(window["date"] == target_day) & window["pm25"].isna()]

    for _, row in day_rows.iterrows():
        sensor_id = row["sensor_id"]
        if sensor_id not in model_cache:
            model, saved_model_dir, _ = retrieved_models[sensor_id]
            xgb = XGBRegressor()
            xgb.load_model(Path(saved_model_dir) / "model.json")
            booster = xgb.get_booster()
            model_cache[sensor_id] = (xgb, booster.feature_names or list(row.index))

        xgb, model_features = model_cache[sensor_id]
        features = (row.reindex(model_features).to_frame().T.apply(pd.to_numeric, errors="coerce"))
        y_hat = xgb.predict(features)[0]

        idx = batch_data.index[(batch_data["sensor_id"] == sensor_id) & (batch_data["date"] == target_day)][0]
        batch_data.at[idx, "pm25"] = y_hat
        batch_data.at[idx, "predicted_pm25"] = y_hat
        min_date = batch_data.loc[batch_data["sensor_id"] == sensor_id, "date"].min()
        batch_data.at[idx, "days_before_forecast_day"] = (target_day - min_date).days + 1

    # recompute features for all sensors now that this days values exist
    temp_df = batch_data.loc[batch_data["date"] <= target_day].copy()
    temp_df = airquality.add_rolling_window_feature(
        temp_df, window_days=3, column="pm25", new_column="pm25_rolling_3d"
    )
    temp_df = airquality.add_lagged_features(temp_df, column="pm25", lags=[1, 2, 3])
    temp_df = airquality.add_nearby_sensor_feature(
        temp_df,
        locations,
        column="pm25",
        n_closest=3,
        new_column="pm25_nearby_avg",
    )

    current_rows = temp_df[temp_df["date"] == target_day]
    for _, row in current_rows.iterrows():
        sensor_id = row["sensor_id"]
        mask = (batch_data["sensor_id"] == sensor_id) & (batch_data["date"] == target_day)
        if mask.any():
            for col in feature_cols:
                batch_data.loc[mask, f"predicted_{col}"] = row[col]

predictions = batch_data.loc[
    batch_data["predicted_pm25"].notna(),
    ["date", "sensor_id", "predicted_pm25", "days_before_forecast_day"]
    + [f"predicted_{col}" for col in feature_cols],
].reset_index(drop=True)

In [10]:
forecast_paths = []

for sensor_id, location in locations.items():
    sensor_forecast = predictions[predictions["sensor_id"] == sensor_id].copy()

    city, street = location["city"], location["street"]
    forecast_path = f"{root_dir}/models/{sensor_id}/images/forecast.png"
    Path(forecast_path).parent.mkdir(parents=True, exist_ok=True)

    plt = airquality.plot_air_quality_forecast(
        location["city"],
        location["street"],
        sensor_forecast,
        forecast_path,
        hindcast=False,
    )
    plt.close()
    forecast_paths.append((sensor_id, forecast_path))

dataset_api = project.get_dataset_api()
str_today = today.strftime("%Y-%m-%d")
if not dataset_api.exists("Resources/airquality"):
    dataset_api.mkdir("Resources/airquality")

for sensor_id, forecast_path in forecast_paths:
    dataset_api.upload(
        forecast_path,
        f"Resources/airquality/{sensor_id}_{str_today}_forecast.png",
        overwrite=True,
    )
print(f"Forecast plots available in Hopsworks under {project.get_url()}/settings/fb/path/Resources/airquality")

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/154549/images/forecast.png: 0.000%|       …

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/60541/images/forecast.png: 0.000%|        …

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/79750/images/forecast.png: 0.000%|        …

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/404209/images/forecast.png: 0.000%|       …

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/59095/images/forecast.png: 0.000%|        …

...

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/69724/images/forecast.png: 0.000%|        …

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/61714/images/forecast.png: 0.000%|        …

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/65146/images/forecast.png: 0.000%|        …

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/69628/images/forecast.png: 0.000%|        …

Forecast plots available in Hopsworks under https://c.app.hopsworks.ai:443/p/1279179/settings/fb/path/Resources/airquality


In [11]:
# Insert predictions into monitoring feature group
monitor_fg = fs.get_or_create_feature_group(
    name="aq_predictions",
    description="Air Quality prediction monitoring",
    version=1,
    primary_key=["sensor_id", "date", "days_before_forecast_day"],
    event_time="date",
)
monitor_fg.insert(predictions, wait=True)


Uploading Dataframe: 100.00% |██████████| Rows 93/93 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: aq_predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279179/jobs/named/aq_predictions_1_offline_fg_materialization/executions
2025-11-17 17:29:01,370 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-17 17:29:07,734 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-17 17:32:27,095 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-17 17:32:27,270 INFO: Waiting for log aggregation to finish.
2025-11-17 17:32:35,869 INFO: Execution finished successfully.


(Job('aq_predictions_1_offline_fg_materialization', 'SPARK'), None)

## Prediction Hindcast: Comparing predicted with forecasted values (1-day prior forecast)


In [12]:

monitoring_df = monitor_fg.filter(monitor_fg.days_before_forecast_day == 1).read()
monitoring_df["date"] = pd.to_datetime(monitoring_df["date"]).dt.tz_localize(None)

air_quality_df = air_quality_fg.read()[["date", "sensor_id", "pm25"]]
air_quality_df["date"] = pd.to_datetime(air_quality_df["date"]).dt.tz_localize(None)

for sensor_id, location in locations.items():
    sensor_preds = monitoring_df[monitoring_df["sensor_id"] == sensor_id][["date", "predicted_pm25"]]
    merged = sensor_preds.merge(
        air_quality_df[air_quality_df["sensor_id"] == sensor_id][["date", "pm25"]],
        on="date",
        how="inner",
    ).sort_values("date")

    city, street = location["city"], location["street"]
    hindcast_path = f"{root_dir}/models/{sensor_id}/images/hindcast_prediction.png"
    Path(hindcast_path).parent.mkdir(parents=True, exist_ok=True)

    plt = airquality.plot_air_quality_forecast(
        city,
        street,
        merged if not merged.empty else sensor_preds.assign(pm25=np.nan),
        hindcast_path,
        hindcast=True,
    )
    plt.close()

    dataset_api.upload(
        hindcast_path,
        f"Resources/airquality/{sensor_id}_{today:%Y-%m-%d}_hindcast.png",
        overwrite=True,
    )

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.98s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.66s) 


Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/154549/images/hindcast_prediction.png: 0.0…

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/60541/images/hindcast_prediction.png: 0.00…

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/79750/images/hindcast_prediction.png: 0.00…

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/404209/images/hindcast_prediction.png: 0.0…

...

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/59893/images/hindcast_prediction.png: 0.00…

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/69724/images/hindcast_prediction.png: 0.00…

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/61714/images/hindcast_prediction.png: 0.00…

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/65146/images/hindcast_prediction.png: 0.00…

Uploading /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn/models/69628/images/hindcast_prediction.png: 0.00…